**Compute Financial Ratios using WRDS/S&P Compustat

In [ ]:
# Import required libraries 
import wrds
import pandas as pd
import plotly.express as px

**What is Compustat?** 
"Compustat is a comprehensive database of fundamental financial and market information on both active and inactive global companies, indices, and industries." ~ WRDS

**Compustat Cheat Sheet** 

| Compustat Variabl | Description | 
| --- | --- |
|at | Total Assets |
| act | Current Assets |
| lt | Total Liabilities |
| lct | Current Liabilities |
| seq | Total Shareholder's Equity |
| dltt | Long Term Debt |
| dlc | Short Term Debt |
| ni | Net Income |
| sale | Sales |
| cogs | Cost of Goods Sold |
| oibdp | Operating Income Before Depreciation |
| oiadp | Operating Income After Depreciation |
| invt | Inventory |
| ppent | Net Property, Plant, and Equipment |
| wcap | Working capital i.e. Current Assets - Current Liabilities |
| csho | Number of shares outstanding |
| prcc_f | Stock price at the end of the fiscal year |
| re | Retained earnings |

In [ ]:
# Get data from Compustat on WRDS

db = wrds.Connection(wrds_username='username') #Connect to the WRDS server

comp = pd.DataFrame(db.raw_sql("""
                    select gvkey, tic, conm, datadate,  
                    lt, at, lct, act, seq, dltt,dlc, ni, sale, cogs, oibdp, oiadp, invt, ppent, wcap, csho, prcc_f, re, rect, ap
                    from comp.funda
                    where indfmt='INDL' 
                    and datafmt='STD'
                    and popsrc='D'
                    and consol='C'
                    and datadate >= '01/01/2010' and tic in ('AAPL')
                    """))  #Use SQL to download required data from the Compustat database

db.close() #Close connection to the WRDS server


# Inspect data
print(comp)

In [ ]:
# Compute year from datadate
comp['datadate']=pd.to_datetime(comp['datadate']) #convert datadate to date fmt
comp['year']=comp['datadate'].dt.year #get year from datadate

In [ ]:
## Define Financial Metrics

# Leverage Ratios
comp['debt']=comp['dltt'] + comp['dlc'] # TOTAL DEBT = LONG TERM + SHORT TERM DEBT
comp['de_ratio'] = comp['debt']/comp['seq'] # D/E RATIO = TOTAL DEBT / TOTAL SHAREHOLDER'S EQUITY
comp['da_ratio'] = comp['debt']/comp['at'] # D/A RATIO = TOTAL DEBT / TOTAL ASSETS

# Liquidity Ratios 
comp['current_ratio'] = comp['act']/comp['lct'] # CURRENT RATIO = CURRENT ASSETS / CURRENT LIABILITIES
comp['quick_ratio'] = (comp['act']-comp['invt'])/comp['lct'] # QUICK RATIO = (CURRENT ASSETS-INVENTORY) / CURRENT LIABILITIES

# Profitability Ratios 
comp['roa'] = comp['ni']/comp['at'] # ROA = NET INCOME / TOTAL ASSETS
comp['roe'] = comp['ni']/comp['seq'] # ROE = NET INCOME / TOTAL SHAREHOLDER'S EQUITY
comp['gross_margin'] = (comp['sale']-comp['cogs'])/comp['sale'] # GROSS MARGIN = (SALES - COGS)/ SALES
comp['operating_margin'] = comp['oibdp']/comp['sale'] # OPERATING MARGIN = OPERATING INCOME / SALES

# Efficiency Ratios 
comp['lag_invt'] = comp.groupby('tic')['invt'].shift() # Compute Lag Inventory 
comp['avg_invt'] = (comp['invt']+comp['lag_invt'])/2 # Compute Average Inventory

comp['days_in_inventory'] = comp['avg_invt']/(comp['cogs']/360) # DAYS IN INVENTORY = AVERAGE INVENTORY / (COGS/360)
comp['inventory_turns'] = 360/comp['days_in_inventory'] # INVENTORY TURNS = 360 / DAYS IN INVENTORY

comp['days_sales_outstanding'] = comp['rect']/(comp['sale']/360) # DAYS SALES OUTSTANDING =  ACCOUNTS RECEIVABLE / (SALES/360)
comp['days_payable_outstanding'] = comp['ap']/(comp['cogs']/360) # DAYS PAYABLE OUTSTANDING = ACCOUNTS PAYABLE / (COSGS/360)

comp['ppe_turnover'] = comp['sale']/comp['ppent'] # PPE TURNOVER = SALES / PPE 
comp['asset_turnover'] = comp['sale']/comp['at'] # ASSET TURNOVER = SALES / TOTAL ASSETS


In [ ]:
# View last available values
comp.sort_values('datadate').groupby('tic').tail(1)

In [ ]:
# Plot ratio trend 
fig = px.line(comp, x="year",y="roa",color="conm")
fig.show()